In [1]:
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  18987      0 --:--:-- --:--:-- --:--:-- 18987
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hi

In [4]:
!apt-get install tensorflow-model-server


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 210 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.3.0 [210 MB]
Fetched 210 MB in 3s (64.6 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.3.0_all.deb ...
Unpacking tensorflow-model-server (2.3.0) ...
Setting up tensorflow-model-server (2.3.0) ...


In [5]:
os.environ["MODEL_DIR"] = "/content/drive/MyDrive/exported_model/11"


In [6]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=exported_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


Starting job # 0 in a separate thread.


In [7]:
!tail server.log

2020-12-04 22:44:30.611772: W tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:267] No versions of servable exported_model found under base path /content/drive/MyDrive/exported_model/11
2020-12-04 22:44:31.611573: W tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:267] No versions of servable exported_model found under base path /content/drive/MyDrive/exported_model/11
2020-12-04 22:44:32.611640: W tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:267] No versions of servable exported_model found under base path /content/drive/MyDrive/exported_model/11
2020-12-04 22:44:33.611683: W tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:267] No versions of servable exported_model found under base path /content/drive/MyDrive/exported_model/11
2020-12-04 22:44:34.611794: W tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:267] No versions of servable exported_model fou

In [8]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": ["this is the best movie"]})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))


Data: {"signature_name": "serving_default", "instances": ... g_default", "instances": ["this is the best movie"]}


In [9]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/exported_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)

ConnectionError: ignored